In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'


In [ ]:
from pathlib import Path
import logging

import tensorflow as tf
from tensorflow import keras as K
#from tensorflow.python.keras import layers
#from tensorflow.python.keras import models

import nibabel as nib
import numpy as np
import os
import datetime
    
import matplotlib.pyplot as plt

from nncf import NNCFConfig
from nncf.tensorflow.helpers.model_creation import create_compressed_model
from nncf.tensorflow.initialization import register_default_init_args
from nncf.common.utils.logger import set_log_level



In [ ]:
Need to define a data path, original model path, and all created model paths

tensorflow model in fp32 (h5 & pb)

tensorflow + mo in various data types

tensorflow + pot in various data types

tensorflow + nncf 

tensorflow + nncf + mo in various datatypes

In [ ]:
models
    base_model_name + framework + datatype
    base_model_name + framework + openvino + datatype
    base_model_name + framework + openvino + pot + datatype
    base_model_name + framework + nncf + datatype
    base_model_name + framework + nncf + openvino + datatype

    

In [ ]:
set_log_level(logging.ERROR)

MODEL_DIR = Path("/home/ubuntu/unet/3D/models")
OUTPUT_DIR = Path("/home/ubuntu/unet/3D/models/openvino")


MODEL_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

BASE_MODEL_NAME = "3d_unet_decathlon"


# Path of baseline TF model with FP32 precision
fp32_h5_path = Path(MODEL_DIR / BASE_MODEL_NAME / BASE_MODEL_NAME).with_suffix(".h5")
fp32_sm_path = Path(MODEL_DIR / BASE_MODEL_NAME / "saved_model.pb")

# Path of optimized TF model using OpenVINO Model Optimizer with FP32 precision
fp32_ir_name = Path(BASE_MODEL_NAME + "_" + "tf" + "_" + "ov" + "_" + "fp32" + "_ir").with_suffix(".xml")
fp32_ir_path = Path(OUTPUT_DIR / fp32_ir_name)

# Path of compressed TF model using OpenVINO Neural Net Compression Framework with INT8 precision
int8_pb_folder = Path(BASE_MODEL_NAME + "_" + "tf" + "_" + "nncf" + "_" + "int8")#.with_suffix(".pb")
int8_pb_path = Path(OUTPUT_DIR / int8_pb_folder)
int8_pb_path.mkdir(parents=True, exist_ok=True)
int8_pb_file = Path(int8_pb_path / "saved_model").with_suffix(".pb")


# Path of finetuned, compressed TF model using OpenVINO Neural Net Compression Framework with INT8 precision
int8_ft_pb_folder = Path(BASE_MODEL_NAME + "_" + "tf" + "_" + "nncf" + "_" + "ft_"+ "int8")#.with_suffix(".pb")
int8_ft_pb_path = Path(OUTPUT_DIR / int8_ft_pb_folder)
int8_ft_pb_path.mkdir(parents=True, exist_ok=True)
int8_ft_pb_file = Path(int8_ft_pb_path / "saved_model").with_suffix(".pb")

# # Path of finetuned, compressed TF model using OpenVINO Neural Net Compression Framework with INT8 precision
# int8_ft_pb_name = Path(BASE_MODEL_NAME + "_" + "tf" + "_" + "nncf" + "_" + "ft_"+ "int8").with_suffix(".pb")
# int8_ft_pb_path = Path(OUTPUT_DIR / int8_ft_pb_name)


# Path of optimized, compressed TF model using OpenVINO Neural Net Compression Framework then Model Optimizer with INT8 precision
int8_ir_name = Path(BASE_MODEL_NAME + "_" + "tf" + "_" + "nncf" + "_" + "ov" + "_" + "int8" + "_ir").with_suffix(".xml")
int8_ir_path = Path(OUTPUT_DIR / int8_ir_name)


In [ ]:
data_path="/home/ubuntu/unet/data/Task01_BrainTumour/"
train_test_split = 0.80
validate_test_split = 0.50
batch_size_train = 8
batch_size_validate = 4
batch_size_test = 1
crop_dim = (128,128,128,1)
#crop_dim = (144,144,144,1)

tile_height = 128
tile_width = 128
tile_depth = 128
number_input_channels = 1

number_output_classes = 1
random_seed = 64

filters = 8
#saved_model_name = "3d_unet_decathlon"


num_epochs=30


In [ ]:
def get_mkl_enabled_flag():

    mkl_enabled = False
    major_version = int(tf.__version__.split(".")[0])
    minor_version = int(tf.__version__.split(".")[1])
    if major_version >= 2:
        if minor_version < 5:
            from tensorflow.python import _pywrap_util_port
        elif minor_version >= 9:

            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '1'))

        else:
            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '0'))
        mkl_enabled = _pywrap_util_port.IsMklEnabled() or (onednn_enabled == 1)
    else:
        mkl_enabled = tf.pywrap_tensorflow.IsMklEnabled()
    return mkl_enabled

print ("We are using Tensorflow version", tf.__version__)
print("MKL enabled :", get_mkl_enabled_flag())

In [ ]:
from dataloader import DatasetGenerator

In [ ]:
brats_datafiles = DatasetGenerator(data_path=data_path, 
                                   train_test_split=train_test_split,
                                   validate_test_split=validate_test_split,
                                   batch_size_train=batch_size_train,
                                   batch_size_validate=batch_size_validate,
                                   batch_size_test=batch_size_test,
                                   tile_height=tile_height, 
                                   tile_width=tile_width, 
                                   tile_depth=tile_depth, 
                                   number_input_channels=number_input_channels,
                                   number_output_classes=number_output_classes,
                                   random_seed=random_seed)
brats_datafiles.print_info()

In [ ]:
# Load and complie the baseline TF model with FP32 precision
from model import dice_coef, soft_dice_coef, dice_loss
tf_baseline_model_fp32 = tf.keras.models.load_model(fp32_h5_path, 
                                      compile=False, 
                                      custom_objects={"dice_coef":dice_coef, "soft_dice_coef":soft_dice_coef, "dice_loss":dice_loss})
tf_baseline_model_fp32.compile(loss=dice_loss, optimizer="adam", metrics=[dice_coef, soft_dice_coef])

In [ ]:
# Validate the model
tf_baseline_fp32_loss, tf_baseline_fp32_dice_coef, tf_baseline_fp32_soft_dice_coef = tf_baseline_model_fp32.evaluate(brats_datafiles.get_test())
print(f"\nLoss of TF Baseline FP32 model: {tf_baseline_fp32_loss:.3f}\nDice Coef of FP32 model: {tf_baseline_fp32_dice_coef:.3f}\nSoft Dice Coef of FP32 model: {tf_baseline_fp32_soft_dice_coef:.3f}")

In [ ]:
nncf_config_dict = {
    "input_info": {"sample_size": [1, number_input_channels, tile_height, tile_width, tile_depth]},
    "log_dir": str(OUTPUT_DIR),  # The log directory for NNCF-specific logging outputs.
    "compression": {
        "algorithm": "quantization",  # Specify the algorithm here.
    },
}
nncf_config = NNCFConfig.from_dict(nncf_config_dict)

In [ ]:
nncf_config = register_default_init_args(nncf_config=nncf_config,
                                         data_loader=brats_datafiles.get_train(),
                                         batch_size=batch_size_train)


In [ ]:
compression_ctrl, compressed_tf_model_int8 = create_compressed_model(tf_baseline_model_fp32, nncf_config)

In [ ]:
compression_ctrl.export_model(int8_pb_file, 'frozen_graph')
print(f'Absolute path where the int8 model is saved:\n {int8_pb_file.resolve()}')

In [ ]:
test_nncf = tf.keras.models.load_model(int8_pb_path, 
                                      compile=False, 
                                      custom_objects={"dice_coef":dice_coef, "soft_dice_coef":soft_dice_coef, "dice_loss":dice_loss})
test_nncf.compile(loss=dice_loss, optimizer="adam", metrics=[dice_coef, soft_dice_coef])

# Validate the INT8 model.
test_nncf_loss, test_nncf_dice_coef, test_nncf_soft_dice_coef = test_nncf.evaluate(brats_datafiles.get_test())
print(f"\nLoss of test_nncf INT8 model: {test_nncf_loss:.3f}\nDice Coef of test_nncf INT8 model: {test_nncf_dice_coef:.3f}\nSoft Dice Coef of test_nncf INT8 model: {test_nncf_soft_dice_coef:.3f}")


In [ ]:
# Compile the INT8 model.
compressed_tf_model_int8.compile(optimizer="adam", 
              loss=dice_loss,
              metrics=[dice_coef, soft_dice_coef])

# Validate the INT8 model.
compressed_tf_model_int8_loss, compressed_tf_model_int8_dice_coef, compressed_tf_model_int8_soft_dice_coef = compressed_tf_model_int8.evaluate(brats_datafiles.get_test())
print(f"\nLoss of INT8 model: {compressed_tf_model_int8_loss:.3f}\nDice Coef of INT8 model: {compressed_tf_model_int8_dice_coef:.3f}\nSoft Dice Coef of INT8 model: {compressed_tf_model_int8_soft_dice_coef:.3f}")


In [ ]:
# Train the INT8 model.
steps_per_epoch = brats_datafiles.num_train // batch_size_train

compressed_tf_model_int8.fit(brats_datafiles.get_train(),
          steps_per_epoch=steps_per_epoch,
          epochs=2)

# Validate the INT8 model.
compressed_ft_tf_model_int8_loss, compressed_ft_tf_model_int8_dice_coef, compressed_ft_tf_model_int8_soft_dice_coef  = compressed_tf_model_int8.evaluate(brats_datafiles.get_test())



In [ ]:
# print(f"\nAccuracy drop of tuned INT8 model over pre-trained FP32 model: {acc_fp32 - acc_int8:.3f}")
print(f"Loss of FP32 model: {tf_baseline_fp32_loss:.3f}\nDice Coef of FP32 model: {tf_baseline_fp32_dice_coef:.3f}\nSoft Dice Coef of FP32 model: {tf_baseline_fp32_soft_dice_coef:.3f}")
print(f"\nLoss of INT8 model: {compressed_tf_model_int8_loss:.3f}\nDice Coef of INT8 model: {compressed_tf_model_int8_dice_coef:.3f}\nSoft Dice Coef of INT8 model: {compressed_tf_model_int8_soft_dice_coef:.3f}")
print(f"\nLoss of Finetuned INT8 model: {compressed_ft_tf_model_int8_loss:.3f}\nDice Coef of Finetuned INT8 model: {compressed_ft_tf_model_int8_dice_coef:.3f}\nSoft Dice Coef of Finetuned INT8 model: {compressed_ft_tf_model_int8_soft_dice_coef:.3f}")


In [ ]:
compressed_tf_model_int8.save(int8_ft_pb_path)
print(f'Absolute path where the model is saved:\n {int8_ft_pb_path.resolve()}')

In [ ]:
from model import dice_coef, soft_dice_coef, dice_loss

test_nncf_ft = tf.keras.models.load_model(int8_ft_pb_path, 
                                      compile=False, 
                                      custom_objects={"dice_coef":dice_coef, "soft_dice_coef":soft_dice_coef, "dice_loss":dice_loss})
test_nncf_ft.compile(loss=dice_loss, optimizer="adam", metrics=[dice_coef, soft_dice_coef])

# Validate the INT8 model.
test_nncf_ft_loss, test_nncf_ft_dice_coef, test_nncf_ft_soft_dice_coef = test_nncf_ft.evaluate(brats_datafiles.get_test())
print(f"\nLoss of test_nncf INT8 model: {test_nncf_ft_loss:.3f}\nDice Coef of test_nncf INT8 model: {test_nncf_ft_dice_coef:.3f}\nSoft Dice Coef of test_nncf INT8 model: {test_nncf_ft_soft_dice_coef:.3f}")


In [ ]:
# !mo -h

In [ ]:
# !mo --framework=tf --input_shape=[1,128,128,128,1] --data_type "FP32" --input=data --input_model=$fp32_sm_path --output_dir=$OUTPUT_DIR


In [ ]:
!mo --framework=tf --input_shape=[1,$tile_height,$tile_width,$tile_depth,$number_input_channels] --data_type "FP32" --input_model=$int8_ft_pb_path --output_dir=$OUTPUT_DIR


In [ ]:
# !mo --framework=tf --input_shape=[1,128,128,128,1] --input=Placeholder --input_model=$int8_pb_path --output_dir=$OUTPUT_DIR

In [ ]:
openvino_filename = "/home/ubuntu/unet/3D/3d_unet_decathlon/NNCF/output/3d_unet_int8"
path_to_xml_file = f"{openvino_filename}.xml"
path_to_bin_file = f"{openvino_filename}.bin"

ie = Core()
model_int8 = ie.read_model(model=path_to_xml_file, weights=path_to_bin_file)
compiled_model_int8 = ie.compile_model(model=modelmodel_int8, device_name="CPU")

del model_int8

input_layer = next(iter(compiled_model_int8.inputs))
output_layer = next(iter(compiled_model_int8.outputs))

In [ ]:
def parse_benchmark_output(benchmark_output):
    parsed_output = [line for line in benchmark_output if not (line.startswith(r"[") or line.startswith("  ") or line == "")]
    print(*parsed_output, sep='\n')


print('Benchmark FP32 model (IR)')
benchmark_output = ! benchmark_app -m $fp32_ir_path -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)

print('\nBenchmark INT8 model (IR)')
benchmark_output = ! benchmark_app -m $int8_ir_path -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)


In [ ]:
from openvino.runtime import Core

ie = Core()
ie.get_property('CPU', "FULL_DEVICE_NAME")